# Surrogate Construction for Genz Functions

This notebook constructs a PC surrogate for Genz functions using full and sparse quadrature and gives the root mean square error between the surrogate and the actual function.

In [1]:
from __future__ import print_function
import numpy as np
import math  

import PyUQTk.pce as uqtkpce
import PyUQTk.PyPCE.pce_tools as pce_tools
from PyUQTk.utils.func import *

PyMC is required for some of the MCMC postprocessing codes.
Will proceed without, but some convergence tests will not be available.


## Inputs

In [2]:
nord = 7            # Order of the PCE
pc_type = "LU"      # Polynomial type
pc_alpha = 0.0      # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
pc_beta = 1.0       # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
param = nord +1     # Number of quadrature points per dimension for full quadrature or level for sparse quadrature
nSam = 100000       # Number of random samples
ndim = 5            # Number of dimensions
model= 'genz_exp'   # Choices are 'genz_osc', 'genz_exp', 'genz_cont','genz_gaus','genz_cpeak', 'genz_ppeak'

## Full Quadrature

First, we instantiate the PC object with full quadrature.

In [3]:
pc_model = uqtkpce.PCSet("NISPnoq", nord, ndim, pc_type, pc_alpha,pc_beta)
pc_model.SetQuadRule(pc_type, 'full', param)

Then, we get a NumPy array of quadrature points, qdpts. Totquat is the total number of quadrature points.

In [4]:
qdpts, totquat= pce_tools.UQTkGetQuadPoints(pc_model)

We evaluate the forward model, the chosen Genz function. The input parameters for the model are all 1, for simpicity, and must be in the form of an array of dimension ndim+1, where the first entry is the shift.
We obtain f_evals, an array of evaluations.

In [5]:
f_evals=func(qdpts,model,np.ones(ndim+1))

The shape of ww is:  ()
The shape of gcf is:  (5,)
The shape of x is:  (32768, 5)


We use Galerkin projection to obtain the PC coefficents.

In [6]:
c_k = pce_tools.UQTkGalerkinProjection(pc_model,f_evals)

We then generate germ samples at which to evaluate the PCE. These are the points at which we are checking the accuracy of the surrogate.

In [7]:
germ_samples=np.random.normal(0,1, (nSam,ndim))
pce_evals=pce_tools.UQTkEvaluatePCE(pc_model,c_k,germ_samples)

We find the actual values of the model at each germ sample and use that to calculate the RMS error between the surrogate and model.

In [8]:
f_actual=func(germ_samples,model,np.ones(ndim+1))
MSE = np.square(np.subtract(f_actual,pce_evals)).mean()
RMSE=math.sqrt(MSE)

print("The RMS error between a", ndim, "-dimensional", model, "function and a full PC surrogate of order", nord, "is")
RMSE

The shape of ww is:  ()
The shape of gcf is:  (5,)
The shape of x is:  (100000, 5)
The RMS error between a 5 -dimensional genz_exp function and a full PC surrogate of order 7 is


0.004579877365484132

## Sparse Quadrature
We repeat the process with sparse quadrature.

In [9]:
pc_model2 = uqtkpce.PCSet("NISPnoq", nord,ndim,pc_type, pc_alpha,pc_beta)
pc_model2.SetQuadRule(pc_type, 'sparse', param)
qdpts2, totquat2= pce_tools.UQTkGetQuadPoints(pc_model2)
f_evals2=func(qdpts2,model,np.ones(ndim+1))
c_k2 = pce_tools.UQTkGalerkinProjection(pc_model2,f_evals2)
germ_samples2=np.random.normal(0,1, (nSam,ndim))
pce_evals2=pce_tools.UQTkEvaluatePCE(pc_model2,c_k2,germ_samples2)
MSE2 = np.square(np.subtract(f_actual,pce_evals2)).mean()
RMSE2=math.sqrt(MSE2)
print("The RMS error between a", ndim, "-dimensional", model, "function and a sparse PC surrogate of order", nord, "is")
RMSE2

The shape of ww is:  ()
The shape of gcf is:  (5,)
The shape of x is:  (187903, 5)
The RMS error between a 5 -dimensional genz_exp function and a sparse PC surrogate of order 7 is


0.3407980253345635

## Monte Carlo Sampling

For Monte Carlo Sampling

The shape of ww is:  ()
The shape of gcf is:  (5,)
The shape of x is:  (100000, 5)
The shape of ww is:  ()
The shape of gcf is:  (5,)
The shape of x is:  (100000, 5)
The shape of ww is:  ()
The shape of gcf is:  (5,)
The shape of x is:  (100000, 5)
The shape of ww is:  ()
The shape of gcf is:  (5,)
The shape of x is:  (100000, 5)
The shape of ww is:  ()
The shape of gcf is:  (5,)
The shape of x is:  (100000, 5)
The shape of ww is:  ()
The shape of gcf is:  (5,)
The shape of x is:  (100000, 5)
The shape of ww is:  ()
The shape of gcf is:  (5,)
The shape of x is:  (100000, 5)
The shape of ww is:  ()
The shape of gcf is:  (5,)
The shape of x is:  (100000, 5)
The shape of ww is:  ()
The shape of gcf is:  (5,)
The shape of x is:  (100000, 5)
The shape of ww is:  ()
The shape of gcf is:  (5,)
The shape of x is:  (100000, 5)
Over 10 samples, the average RMS error between a 5 -dimensional genz_exp function and MC sampling is


0.2554013850179419